In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import scipy.io as sio
import os
import ieeg_funcs as ief
import dgFuncs as dg
from sklearn import svm
from sklearn.externals import joblib
import 

In [2]:
# Load model
temp_model_fname='/Users/davidgroppe/PycharmProjects/SZR_ANT/MODELS/model_sbox/model0.pkl'
temp_model=joblib.load(temp_model_fname)

In [6]:
# Get list of models
model_name='model_sbox'
path_dict=ief.get_path_dict()
model_path=os.path.join(path_dict['szr_ant_root'],'MODELS',model_name)
model_list=[]
for f in os.listdir(model_path):
    if f.endswith('.pkl'):
        model_list.append(f)

        
n_models=len(model_list)
print('%d models found' % n_models)
print(model_list)

8 models found
['model0.pkl', 'model1.pkl', 'model2.pkl', 'model3.pkl', 'model4.pkl', 'model5.pkl', 'model6.pkl', 'model7.pkl']


In [8]:
# Import held sub data
sub='NA'

# Get list of features to use:
ftr_types=[]
ftr_fname='ftr_list_lap.txt'
print('Importing list of features to use from %s' % ftr_fname)
text_file = open(ftr_fname, 'r')
list1 = text_file.readlines()
for temp_ftr in list1:
    ftr_types.append(temp_ftr.rstrip())
print('Features being used: {}'.format(ftr_types))


Importing list of features to use from ftr_list_lap.txt
Features being used: ['PWR', 'PWR_3SEC', 'VLTG']


In [14]:
# Get list of data files for this sub
n_ftr_types=len(ftr_types)
n_dim=0
n_wind_list=[]
stem_list=[]
for type_ct, ftr_type in enumerate(ftr_types):
    print('Checking dimensions of feature type %s' % ftr_type)
    # Figure out how much data there is to preallocate mem
    ftr_type_dir=os.path.join(path_dict['ftrs_root'],ftr_type)

    if type_ct==0:
        # count time windows and create fname_stem_list
        stem_list = []
        ftr_path=os.path.join(ftr_type_dir,sub)
        for f in os.listdir(ftr_path):
            #get file stem
            f_parts = f.split('_')
            f_stem = f_parts[0]+'_'+f_parts[1]+'_'+f_parts[2]
            #fname_stem_list.append(f_stem)
            stem_list.append(f_stem)
            ftr_dict=np.load(os.path.join(ftr_path,f))
            #n_wind+=np.sum(ftr_dict['peri_ictal']>=0)
            n_wind_list.append(ftr_dict['ftrs'].shape[1])
    else:
        temp_n_wind=0
        f_ct=0
        # Load file extension
        ext_fname = os.path.join(ftr_type_dir, 'ext.txt')
        with open(ext_fname, 'r') as f_ext:
            ext = f_ext.readline()[:-1]
        print(ext_fname)
        print(ext)
        ftr_path=os.path.join(ftr_type_dir,sub)
        for temp_stem_ct, temp_stem in enumerate(stem_list):
            targ_file=os.path.join(ftr_path,temp_stem+ext)
            # if os.path.isfile(targ_file)==False:
            #     print('File not found: %s' % targ_file)
            #     raise ValueError('File stems do not match across features')
            print('Loading file %s' % targ_file)
            ftr_dict=np.load(targ_file)
            f_ct+=1
        if n_wind_list[temp_stem_ct]!=ftr_dict['ftrs'].shape[1]:
            raise ValueError('# of time windows do not match across features')
    n_dim += ftr_dict['ftrs'].shape[0]

print('Total # of dimensions: %d ' % n_dim)
print('Total # of time windows: {} '.format(n_wind_list))
print('Total # of files: %d' % f_ct)


Checking dimensions of feature type PWR
Checking dimensions of feature type PWR_3SEC
/Users/davidgroppe/PycharmProjects/SZR_ANT/FTRS/PWR_3SEC/ext.txt
_bppwr_3sec.npz
Loading file /Users/davidgroppe/PycharmProjects/SZR_ANT/FTRS/PWR_3SEC/NA/NA_d1_sz2_bppwr_3sec.npz
Loading file /Users/davidgroppe/PycharmProjects/SZR_ANT/FTRS/PWR_3SEC/NA/NA_d1_sz3_bppwr_3sec.npz
Loading file /Users/davidgroppe/PycharmProjects/SZR_ANT/FTRS/PWR_3SEC/NA/NA_d1_sz4_bppwr_3sec.npz
Loading file /Users/davidgroppe/PycharmProjects/SZR_ANT/FTRS/PWR_3SEC/NA/NA_d1_sz5_bppwr_3sec.npz
Loading file /Users/davidgroppe/PycharmProjects/SZR_ANT/FTRS/PWR_3SEC/NA/NA_d2_sz1_bppwr_3sec.npz
Checking dimensions of feature type VLTG
/Users/davidgroppe/PycharmProjects/SZR_ANT/FTRS/VLTG/ext.txt
_vltg.npz
Loading file /Users/davidgroppe/PycharmProjects/SZR_ANT/FTRS/VLTG/NA/NA_d1_sz2_vltg.npz
Loading file /Users/davidgroppe/PycharmProjects/SZR_ANT/FTRS/VLTG/NA/NA_d1_sz3_vltg.npz
Loading file /Users/davidgroppe/PycharmProjects/SZR_ANT/

In [15]:
# Load validation data and calculate false positive rate, and peri-onset latency
onset_dif_sec_list=list()
n_valid_szrs=0
n_missed_szrs=0
mn_onset_dif=0       
for stem_loop in stem_list:
    # Collect features for each seizure
    n_valid_szrs+=1
    dim_ct=0
    for ftr_type in ftr_types:
        ftr_path = os.path.join(path_dict['ftrs_root'], ftr_type, sub)
        file_found=False
        for f_valid in os.listdir(ftr_path):
            # get file stem
            f_parts = f_valid.split('_')
            temp_f_stem = f_parts[0] + '_' + f_parts[1] + '_' + f_parts[2]
            if temp_f_stem==stem_loop:
                # load file
                #print('Loading %s' % f)
                print('Loading %s' % os.path.join(ftr_path, f_valid))
                ftr_dict = np.load(os.path.join(ftr_path, f_valid))
                file_found=True
                # break out of for loop
                break
        # Catch if new file was not loaded
        if not file_found:
            print('Trying to find %s for %s' % (stem_loop,ftr_type))
            print('Looking in dir %s' % ftr_path)
            raise ValueError('File stem not found')
            exit()
        # Add ftr to collection
        temp_n_dim = ftr_dict['ftrs'].shape[0]
        # Note that we use all time points (even ictal points long after ictal onset
        if ftr_type==ftr_types[0]:
            # First feature being analyzed pre-allocate mem
            temp_n_wind=ftr_dict['ftrs'].shape[1]
            temp_valid_ftrs=np.zeros((temp_n_wind,n_dim))
        temp_valid_ftrs[:, dim_ct:dim_ct + temp_n_dim] = ftr_dict['ftrs'].T
        dim_ct += temp_n_dim

    # Classify each time point
    temp_class_hat=rbf_svc.predict(temp_valid_ftrs)
    #temp_class_hat=rbf_svc.predict(temp_valid_ftrs)
    
    # Compute latency of earliest ictal prediction relative to clinician onset
    sgram_srate=1/10
    onset_dif_sec=ief.cmpt_postonset_stim_latency(temp_class_hat,ftr_dict['peri_ictal'],sgram_srate)
    if onset_dif_sec is None:
        # no positives during peri-onset time window
        n_missed_szrs+=1
    else:
        mn_onset_dif+=onset_dif_sec

pcnt_missed_szrs[left_out_id,C_ct] = n_missed_szrs/n_valid_szrs
if n_missed_szrs==n_valid_szrs:
    mn_stim_latency[left_out_id,C_ct] = np.nan
else:
    mn_stim_latency[left_out_id, C_ct] = mn_onset_dif/(n_valid_szrs-n_missed_szrs)

Loading /Users/davidgroppe/PycharmProjects/SZR_ANT/FTRS/PWR/NA/NA_d1_sz2_bppwr.npz
Loading /Users/davidgroppe/PycharmProjects/SZR_ANT/FTRS/PWR_3SEC/NA/NA_d1_sz2_bppwr_3sec.npz
Loading /Users/davidgroppe/PycharmProjects/SZR_ANT/FTRS/VLTG/NA/NA_d1_sz2_vltg.npz


NameError: name 'rbf_svc' is not defined